<a href="https://colab.research.google.com/github/SL-LAIDLAW/projet-outilBioinfo/blob/dev/Projet_Outils_Bioinfo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projet de HMSN204
_Sean LAIDLAW_, _Mathieu Blaison_

### TODO: diagramme de Gantt qui fera une synth`ese des tˆaches effectu´ees et de l’implication de chaque ´etudiant.

Liste des taches et dates (pour faire Gantt après)


*  creation du document colab &  planification du projet = mercredi 27 mars
*  requete NCBI pour sequences sex1 = Samedi, 06 avril  2019 + lundi 08/apr
* writing biopython alignment script mardi 14h-19:40 + 15:40 - 17:00 mercredi
* fixing last minute biopython bugs 01-04am jeudi

## Traitements Biopython de SEX1

### Obtention des sequences sauvages et mutés, du NCBI

#### Obtention du sequence sauvage

In [0]:
!pip install biopython
!pip install pandas
from Bio import SeqIO 
from Bio import Entrez

À partir de TAIR, on a obtenu l'identifiant correspondant au *full length cDNA* de la sequence genomique de SEX1. Un requete envers la base de donnée nucleotide du NCBI nous a fourni le fasta de cette sequence.

In [2]:
Entrez.email = "sean.laidlaw@etu.umontpellier.fr"
seqgb  = Entrez.efetch (db="nucleotide",id='NM_001331926.1', rettype='fasta')
seqselect = SeqIO.parse(seqgb, "fasta")

sex1_sauvage = []
for sequence in seqselect:
    sex1_sauvage = sequence.seq
sex1_sauvage

Seq('TATCTTCTGCAATTCGTGGTTTGTGAACACAAGAGAAAAAAAAAAAAAAACATA...AAG', SingleLetterAlphabet())

#### Obtention des sequences mutés

In [3]:
search_string = "SEX1[Gene] AND Arabidopsis thaliana[Organism]"
req = Entrez.esearch(db= "nucleotide", term=search_string)  # ID of fasta of SEX1 gene
res = Entrez.read(req)
res

DictElement({'Count': '5', 'RetMax': '5', 'RetStart': '0', 'IdList': ['1063682343', '1063682341', '334182442', '240254421', '332189094'], 'TranslationSet': [DictElement({'From': 'Arabidopsis thaliana[Organism]', 'To': '"Arabidopsis thaliana"[Organism]'}, attributes={})], 'TranslationStack': [DictElement({'Term': 'SEX1[Gene]', 'Field': 'Gene', 'Count': '8', 'Explode': 'N'}, attributes={}), DictElement({'Term': '"Arabidopsis thaliana"[Organism]', 'Field': 'Organism', 'Count': '2696552', 'Explode': 'Y'}, attributes={}), 'AND'], 'QueryTranslation': 'SEX1[Gene] AND "Arabidopsis thaliana"[Organism]'}, attributes={})

Ici on a choisi de prendre uniquement le premier resultat de la liste, car les autres fasta ne sont que des variants d'epissage alternative, et donc n'aligneront pas avec les sequences nos mutants.

In [0]:
sex1_id = res["IdList"]
req.close()

In [5]:
seqgb  = Entrez.efetch (db="nucleotide",id=sex1_id, rettype='fasta')
seqselect = SeqIO.parse(seqgb, "fasta")

sex1_mutants = []
for sequence in seqselect:
    sex1_mutants.append(sequence.seq)
sex1_mutants

[Seq('CACAGATACCGTCCATCAACGTATCACTGGGCTCTCTTTCTCCGTGCCGTTCCT...ATT', SingleLetterAlphabet()),
 Seq('TATCTTCTGCAATTCGTGGTTTGTGAACACAAGAGAAAAAAAAAAAAAAACATA...AAG', SingleLetterAlphabet()),
 Seq('TATCTTCTGCAATTCGTGGTTTGTGAACACAAGAGAAAAAAAAAAAAAAACATA...TGG', SingleLetterAlphabet()),
 Seq('CCCTAAACCCTAAACCCTAAACCCTAAACCTCTGAATCCTTAATCCCTAAATCC...GGG', SingleLetterAlphabet()),
 Seq('CCCTAAACCCTAAACCCTAAACCCTAAACCTCTGAATCCTTAATCCCTAAATCC...GGG', SingleLetterAlphabet())]

## Alignement des séquences associées aux gènes sauvages/mutés

### un alignement par paire global des séquences d’ARNm

Pour aligner nos sequences sauvages avec les mutants nous avons ecrit la fonction AlignSeqs qui effectue un alignement globale en utilisant l'algorithme Needleman–Wunsch.

In [0]:
import pandas as pd

In [0]:
def InitializeTable(SeqX,SeqY):
  # intitialise dataframe holding scores
  df = pd.DataFrame(index=SeqY, columns=SeqX)
  col_names = list(df.keys())
  row_names = list(df.index)

  # initiate table
  df.iloc[0,0] = 0
  for i in range(len(SeqY)):
    df.iloc[i,0] = i * -1  # set row 0 to be -1,-2,-3
  for i in range(len(SeqX)):
    df.iloc[0,i] = i * -1  # set col 0 to be -1,-2,-3
  return df

In [0]:
def PopulateTable(seqX,seqY,df):     
  df_dir = df.copy()
  # populate table
  for y in range(1,len(seqY)):
    for x in range(1,len(seqX)):
      current_score = -1000
      
      match_score = 1
      mismatch_score = -1

      if str(seqX[x]) == str(seqY[y]):
        current_score = match_score
      else:
        current_score = mismatch_score

      df.iloc[y,x] = current_score

      top_left_score = ((df.iloc[y-1,x-1]), '\x5C')
      top_score = ((df.iloc[y-1,x]), '\x7C')
      left_score = ((df.iloc[y,x-1]), '\x3C')

      best_score = top_score[0]
      best_direction = top_score[1]
      score_list = [top_left_score, top_score, left_score]
      for score_item in score_list:
        if int(score_item[0]) > int(best_score):
          best_direction = score_item[1]
          best_score = score_item[0]

        elif int(score_item[0]) == int(best_score):
          if not best_direction.__contains__(score_item[1]):
            best_direction += score_item[1]

      # delete score objects from memory as this caused issues
      for score in score_list:
        del score

      best_score = int(best_score)
      best_score += current_score

      df.iloc[y,x] = best_score
      df_dir.iloc[y,x] = best_direction
  return df, df_dir  

In [0]:
def Traceback(df,df_dir):
    # Start traceback
    my_alignment_x =[]
    my_alignment_y =[]
    bottom_right_y = list(df.shape)[0] -1
    bottom_right_x = list(df.shape)[1] -1

    x = bottom_right_x
    y = x - (bottom_right_x - bottom_right_y)
    while x != 0:
      current_coordinates = [y,x]
      top_coordinates = [y-1,x]
      left_coordinates = [y,x-1]
      top_left_coordinates = [y-1,x-1]

      current_nucleotide_x = df.keys()[x]
      current_nucleotide_y = df.index[y]
      current_direction = df_dir.iloc[y,x]

      if current_direction.__contains__('\\'):
        # move coordinates to top_left
        x = top_left_coordinates[1]
        y = top_left_coordinates[0]
        my_alignment_x.append(current_nucleotide_x)
        my_alignment_y.append(current_nucleotide_y)

      elif current_direction.__contains__('|'):
        # move coordinates to top
        x = top_coordinates[1]
        y = top_coordinates[0]
        my_alignment_x.append('-')
        my_alignment_y.append(current_nucleotide_y)

      elif current_direction.__contains__('<'):
        # move coordinates to left
        x = left_coordinates[1]
        y = left_coordinates[0]
        my_alignment_x.append(current_nucleotide_x)
        my_alignment_y.append('-')
        
      return my_alignment_x, my_alignment_y

In [0]:
def GetVariations(my_alignment_x, my_alignment_y):
  points_de_variations = set()
  for i in range(len(my_alignment_x)):
    x = my_alignment_x[i]
    y = my_alignment_y[i]

    if x != y:
      points_de_variations.add(i)
  return list(points_de_variations)

In [0]:
def AlignSeqs(SeqX,SeqY):
  SeqX = list(" " + SeqX)
  SeqY = list(" " + SeqY)

  match_score = 1
  mismatch_score = -1
  indel_score = -1
  
  penaltyArgs=[match_score, mismatch_score, indel_score]

  df = InitializeTable(SeqX,SeqY)

  df, df_dir = PopulateTableJit(SeqX,SeqY,df)


  my_alignment_x, my_alignment_y = Traceback(df,df_dir)

  my_alignment_x.reverse()
  my_alignment_y.reverse()

  points_de_variations = GetVariations(my_alignment_x, my_alignment_y)


  return my_alignment_x, my_alignment_y, points_de_variations

In [0]:
result_dict = {}

mutant = sex1_mutants[0]
# for mutant in sex1_mutants:
alignment_for_mutant, alignment_for_sauvage, variations = AlignSeqs(mutant, sex1_sauvage)
result_dict[mutant] = [alignment_for_mutant, alignment_for_sauvage, variations]

%store result_dict

### donner les positions des variations sauvage/muté

In [0]:
%store -r result_dict

for key in result_dict.keys():
  result_list = result_dict[key]
  variants = result_list[2]
  for var in variants:
    print(var)

### taux de GC / traduction pour obtenir les séquences protéiques

In [0]:
from Bio.SeqUtils import GC

for result_dict
for mutant in sex1_mutants:
  print(GC(mutant))

In [0]:
coding_dna.translate()

# Base de données

## Reflection sur le type de base de données

On a eu le choix d’un des SGBD vu ce semestre : chado/postgresql (module phenotype), relationnel seul (oracle, postgresqlou mysql), neo4j, couchdb, triplestore jena/rdf. 

On a choisi d'eviter les base de données NoSQL car nous avons plus d'experience avec ceux là. 

Notre choix s'est ainsi  porté sur la base de données Chado. Celle-ci est conçue spécialement pour gérer les représentations complexes de données biologiques.

## Modelisation de la base de données

## Implantation de la base de données et peuplement

In [0]:
from Bio import SeqIO
import pandas as pd

In [0]:
df = pd.read_csv("https://raw.githubusercontent.com/SL-LAIDLAW/projet-outilBioinfo/master/Data_Projet_Entete.csv",sep=";")
df['Mutant'] = False
df['Mutant'].loc[df['Genotype'] != "Col_0"] = True
df.loc[df['Mutant'] == True]

In [0]:
cov = df.cov()
cov